In [38]:
#-*- coding: utf-8 -*-

import xlrd
import xlwt
import os
import time;  # 引入time模块
import traceback
from openpyxl import Workbook
from openpyxl import load_workbook
import openpyxl
from xlutils.copy import copy
from copy import deepcopy 

pattern = xlwt.Pattern() # Create the Pattern 
pattern.pattern = xlwt.Pattern.SOLID_PATTERN # May be: NO_PATTERN, SOLID_PATTERN, or 0x00 through 0x12 
pattern.pattern_fore_colour = 3 # May be: 8 through 63. 0 = Black, 1 = White, 2 = Red, 3 = Green, 4 = Blue, 5 = Yellow, 6 = Magenta, 7 = Cyan, 16 = Maroon, 17 = Dark Green, 18 = Dark Blue, 19 = Dark Yellow , almost brown), 20 = Dark Magenta, 21 = Teal, 22 = Light Gray, 23 = Dark Gray, the list goes on...

style = xlwt.XFStyle() # Create the Pattern

style.pattern = pattern # Add Pattern to Style

In [3]:
def fusion_filed_values(ori_list,tar_list):
    keys='-'.join(map(str,ori_list[1:2]))+'-'+str(ori_list[13])
    msg=[]
    point=[]
    for i,x in enumerate(ori_list):
        if tar_list[i]==x:
            pass
        else:
            #print(ori_list)
            #print(tar_list)
            point.append(i)
            msg.append('%s修改:%s->%s'%(keys,str(x),str(tar_list[i])))
            tar_list[i]=x
    return msg,point,tar_list

In [4]:
def comp_filed_values(ori_list,tar_list):

    if ori_list[1]=='':
            #print(ori_list)
        return 'dont'
    
    ori_=[ori_list[1],ori_list[2],ori_list[13],ori_list[22]]
    tar_=[tar_list[1],tar_list[2],tar_list[13],tar_list[22]]
    
    for i,x in enumerate(ori_):
        if x=='':
            ori_.pop(i)
            tar_.pop(i)
    for i,x in enumerate(tar_):
        if x=='':
            ori_.pop(i)
            tar_.pop(i)
            
    if ori_==tar_: 
        return 'done'
    #print(ori_,tar_)
    return 'dont'

In [5]:
def execl_copy(ori_file):
    os.system('copy '+ori_file+' result.xls')

In [6]:
def result_copy(tar_file):
    os.system('copy '+'result.xls '+tar_file)

In [7]:
def append_excel(sheet,rows,line):
    #print(line)
    for col in range(len(line)):
        sheet.write(rows,col,line[col],style)
    return sheet
    

In [9]:
def prn_obj(obj):
    print('\n'.join(['%s:%s' % item for item in obj.__dict__.items()]))

In [50]:
#往日志文件中追加内容函数
def writeappend_logfile(filename,content):
    file=open(filename,'a') #以追加方式打开日志文件
    time_now= time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())  #系统时间格式化
    file.writelines(time_now+':'+content+'\n')      #写入内容
    file.close() #关闭文件

def read_excel(ori_path,tar_path,sub_name):#
    success=0        #匹配一致数量
    fail=0           #匹配不一致数量
    modify=0         #修改的数量
    new=0
    struck_out_num=0
    origin_xls={} #存储源xls文件
    target_xls={} #比对的xls文件
    #execl_copy(ori_path)
    wb_ori=xlrd.open_workbook(ori_path,formatting_info=True) #打开原始文件
    wb_tar=xlrd.open_workbook(tar_path,formatting_info=True) #打开目标文件
    wb_result = copy(wb_tar)  # 新建一个文件，用来保存结果
    sheet_result = wb_result.get_sheet(1)
    

    sheet_num = len(wb_ori.sheets()) #源表子表数量
    
    startime=time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())    #获取系统当前时间并格式化为格式
    print (startime,' 开始比对...')
    logname='log_'+startime[0:10]+'.log'               #截取日期年月日构成日志文件名

    logfile=open(logname,'w')    #创建日志文件,如果文件存在则清空内容，不存在则创建，如果需要同时批量比对多张表，可以考虑将日志文件名作为参数传入
    logfile.writelines(startime+':【开始比对】...'+'\n')       #写入开始时间
    logfile.close()            #关闭日志文件
    
    try:
        sheet_ori=wb_ori.sheet_by_name(sub_name)
        sheet_tar=wb_tar.sheet_by_name(sub_name)
        cinfomap=sheet_ori.colinfo_map
        rinfomap=sheet_ori.rowinfo_map
        tar_cinfomap=sheet_tar.colinfo_map
        tar_rinfomap=sheet_tar.rowinfo_map
        ori_rows, ori_cols = sheet_ori.nrows, sheet_ori.ncols
        tar_rows, tar_cols = sheet_ori.nrows, sheet_ori.ncols

        def get_struck_out(ori_rows,tar_rows):
            # 判断 是否划线了
            t1=0
            t2=0
            if rinfomap.get(ori_rows,0):
                ori_xf = wb_ori.xf_list[rinfomap[ori_rows].xf_index] 
                ori_font = wb_ori.font_list[ori_xf.font_index]
                t1=ori_font.struck_out
            if tar_rinfomap.get(tar_rows,0):
                tar_xf = wb_tar.xf_list[tar_rinfomap[tar_rows].xf_index] 
                tar_font = wb_tar.font_list[tar_xf.font_index]
                t2=tar_font.struck_out
            if t1 > t2:
                print(ori_rows,tar_rows,t1,t2)
                return True
            return False
        print(len(rinfomap))
        for rows in range(1,ori_rows):

            if rinfomap.get(rows,0) and rinfomap[rows].hidden==1:
                continue
            orign_list=sheet_ori.row_values(rows) #源表i行数据
            origin_xls[rows]=orign_list     #源表写入字典
        for rows in range(1,tar_rows):
            if sheet_tar.colinfo_map.get(rows,0) and sheet_tar.colinfo_map[rows].hidden==1:
                continue
            target_list=sheet_tar.row_values(rows) #目标表i行数据
            target_xls[rows]=target_list    #目标表写入字典

        if origin_xls[1]  == target_xls[1]:
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' 表头一致')
        new_add=[]
        for ori_num in origin_xls:
            flag='false'          #判断是否一致标志
            fusion_msg=[]
            temp=style
            for tar_num in target_xls:
                fetch_ret=comp_filed_values( origin_xls[ori_num],target_xls[tar_num])

                if fetch_ret=='done':
                    fusion_msg,point,target_line=fusion_filed_values( origin_xls[ori_num],target_xls[tar_num])
                    flag='true'
                    break              #如果匹配到结果退出循环
            if origin_xls[ori_num][1]!='' and tar_num>=len(target_xls):
                new_add.append(ori_num)
                new+=1
            if flag=='true' and get_struck_out(ori_num,tar_num):
                struck_out_style=xlwt.XFStyle() # Create the Pattern
                struck_out_style.pattern = pattern # Add Pattern to Style
                font = xlwt.Font() 
                font.struck_out = 1
                struck_out_style.font=font
                temp=struck_out_style 
                struck_out_num+=1
                logstr='删除线源行id:{:d},目标行id:{:d}'.format(ori_num,tar_num)
                writeappend_logfile(logname,logstr)
                for x in range(ori_cols):
                    sheet_result.write(tar_num, x, sheet_ori.cell_value(ori_num, x),temp)
            if fusion_msg!=[]:
                writeappend_logfile(logname,','.join(fusion_msg))
                modify+=1
                for x in point:
                    logstr='源行id:{:d},目标行id:{:d},修改列:{:d}'.format(ori_num,tar_num,x)
                    writeappend_logfile(logname,logstr)
                    sheet_result.write(tar_num, x, sheet_ori.cell_value(ori_num, x),temp)
                #print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' row:%d is modify:%s'%(ori_num,','.join(fusion_msg)))

            if flag=='true':           #匹配上结果输出后台日志
                #print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' row:%d is ok'%ori_num)
                success+=1
            else:                      #匹配不上将源表中行记录写入txt
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' row:%d is different'%ori_num)
                fail+=1
                data=origin_xls[ori_num]
                logstr='【不一致】row<'+str(ori_num)+'>:'+str(data)
                writeappend_logfile(logname,logstr)

        if new>0:
            for i,new_line in enumerate(new_add):
                sheet_result=append_excel(sheet_result,tar_rows+i+2,origin_xls[new_line])
       # logstr='【比对完成】总记录数:'+str(ori_num)+'条,一致:'+str(success)+'条,不一致:'+str(fail)+'条'
        #print(list(target_xls.keys()))
        logstr='【比对完成】总记录数:{:d}条,匹配到:{:d}条,变更:{:d}条,未匹配到:{:d}条,新增:{:d}条,删除线:{:d}条'.format(ori_num,
                                                                                 success,modify,fail,new,struck_out_num)
        print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' 【%s】比对结束'%sheet_ori.name)
        print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+' 总记录数:%d条,匹配到:%d条,变更:%d条,未匹配到:%d条,新增:%d条,删除线:%d条'%(ori_num,
                                                                                                        success,modify,fail,new,struck_out_num))
        writeappend_logfile(logname,logstr)
    except Exception as err:
        
        msg=traceback.format_exc()
        print(msg)
        writeappend_logfile(logname,str(msg)) #输出异常
    wb_result.save('result_'+startime[0:10]+'.xls')

    

def main():
    pass

if __name__ == '__main__':

    read_excel(r'b.xls','a.xls','预算目标分解表')

2018-11-30 16:00:31  开始比对...
887
2018-11-30 16:00:31 表头一致
2018-11-30 16:00:31 row:1 is different
694 694 1 0
839 839 1 0
2018-11-30 16:00:31 row:882 is different
2018-11-30 16:00:31 row:883 is different
2018-11-30 16:00:31 row:884 is different
2018-11-30 16:00:31 row:885 is different
2018-11-30 16:00:31 row:886 is different
2018-11-30 16:00:31 row:887 is different
2018-11-30 16:00:31 row:888 is different
2018-11-30 16:00:31 row:889 is different
2018-11-30 16:00:31 row:890 is different
2018-11-30 16:00:31 【预算目标分解表】比对结束
2018-11-30 16:00:31 总记录数:890条,匹配到:59条,变更:4条,未匹配到:10条,新增:0条,删除线:2条


In [ ]:
read_excel(r'b1.xls','a.xls','预算目标分解表')
read_excel(r'b2.xls','result.xls','预算目标分解表')
read_excel(r'b3.xls','result.xls','预算目标分解表')
read_excel(r'b4.xls','result.xls','预算目标分解表')
read_excel(r'b5.xls','result.xls','预算目标分解表')
read_excel(r'b6.xls','result.xls','预算目标分解表')
read_excel(r'b7.xls','result.xls','预算目标分解表')
read_excel(r'b8.xls','result.xls','预算目标分解表')
read_excel(r'b9.xls','result.xls','预算目标分解表')
result.xls 改成a.xls 完事